In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
# Read in 1.2 Million Spotify Songs dataset
spotify_1m = pd.read_csv('./Resources/tracks_features.csv')

In [3]:
# Strip brackets and single quotes from artist name
spotify_1m['artist'] = spotify_1m['artists'].str.strip("[']")

# Preview dataframe
spotify_1m.head()

,id,name,album,album_id,artists,artist_ids,track_number,disc_number,explicit,danceability,...,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year,release_date,artist
0,7lmeHLHBe4nmXzuXc0HDjk,Testify,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],1,1,False,0.470,...,0.02610,0.000011,0.3560,0.503,117.906,210133,4.0,1999,1999-11-02,Rage Against The Machine
1,1wsRitfRRtWyEapl0q22o8,Guerrilla Radio,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],2,1,True,0.599,...,0.01290,0.000071,0.1550,0.489,103.680,206200,4.0,1999,1999-11-02,Rage Against The Machine
2,1hR0fIFK2qRG3f3RF70pb7,Calm Like a Bomb,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],3,1,False,0.315,...,0.02340,0.000002,0.1220,0.370,149.749,298893,4.0,1999,1999-11-02,Rage Against The Machine
3,2lbASgTSoDO7MTuLAXlTW0,Mic Check,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],4,1,True,0.440,...,0.16300,0.000004,0.1210,0.574,96.752,213640,4.0,1999,1999-11-02,Rage Against The Machine
4,1MQTmpYOZ6fcMQc56Hdo7T,Sleep Now In the Fire,The Battle Of Los Angeles,2eia0myWFgoHuttJytCxgX,['Rage Against The Machine'],['2d0hyoQ5ynDBnkvAbJKORj'],5,1,False,0.426,...,0.00162,0.105000,0.0789,0.539,127.059,205600,4.0,1999,1999-11-02,Rage Against The Machine


In [36]:
# Read in Billboard Top 100 dataset
df = pd.read_csv('./Resources/charts.csv')

# Convert date column to date type
df['date'] = pd.to_datetime(df['date'])
# Create new column for year
df['year'] = df['date'].dt.year
# Remove special characters from artist and song name
df['artist'] = df['artist'].str.replace(r"\(.*\)","")
# df['song'] = df['song'].str.replace(r"\(.*\)","")

df['song'] = df['song'].str.replace(".","") \
                        .str.replace("\"", "") \
                        .str.replace(",", "")

# Create new column showing number of weeks in #1 spot (if exists)
df['weeks-at-no1'] = df[df['rank']==1].groupby(['song', 'artist', 'rank'], as_index=False).count()['date']

# Remove single quotes/apostrophes from song names
df['song'] = df['song'].str.replace("'", "")

# Preview dataframe
df.head()

/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  if __name__ == "__main__":
/Users/glena.dagger/Downloads/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  if sys.path[0] == "":


,date,rank,song,artist,last-week,peak-rank,weeks-on-board,year,weeks-at-no1
0,2021-11-06,1,Easy On Me,Adele,1.0,1,3,2021,1.0
1,2021-11-06,2,Stay,The Kid LAROI & Justin Bieber,2.0,1,16,2021,7.0
2,2021-11-06,3,Industry Baby,Lil Nas X & Jack Harlow,3.0,1,14,2021,1.0
3,2021-11-06,4,Fancy Like,Walker Hayes,4.0,3,19,2021,4.0
4,2021-11-06,5,Bad Habits,Ed Sheeran,5.0,2,18,2021,2.0


In [37]:
# Create new dataframe of all unique Billboard charting songs
unique_billboard_tracks_df = df.groupby(['song','artist', 'year'], as_index=False).agg({'peak-rank': 'min', 
                                                                                        'weeks-on-board': 'max',
                                                                                        'weeks-at-no1': 'max'})

# Create separate dataframes for each decade
billboard_1960s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1970) & (unique_billboard_tracks_df['year']>=1960)]
billboard_1970s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1980) & (unique_billboard_tracks_df['year']>=1970)]
billboard_1980s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<1990) & (unique_billboard_tracks_df['year']>=1980)]
billboard_1990s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2000) & (unique_billboard_tracks_df['year']>=1990)]
billboard_2000s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2010) & (unique_billboard_tracks_df['year']>=2000)]
billboard_2010s = unique_billboard_tracks_df[(unique_billboard_tracks_df['year']<2020) & (unique_billboard_tracks_df['year']>=2010)]
billboard_2020s = unique_billboard_tracks_df[unique_billboard_tracks_df['year']>=2020]

# Create list of dataframes
billboard_dfs = [billboard_1960s, billboard_1970s, billboard_1980s, billboard_1990s, billboard_2000s, billboard_2010s, billboard_2020s]

# Display dataframe
unique_billboard_tracks_df.head()
unique_billboard_tracks_df

# df[(df['artist']=='Beastie Boys') & (df['year']==1986)]

,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1
0,#1,Nelly,2001,22,11,NaN
1,#1,Nelly,2002,22,20,NaN
2,#1 Dee Jay,Goody Goody,1978,82,5,NaN
3,#9 Dream,John Lennon,1974,58,2,NaN
4,#9 Dream,John Lennon,1975,9,12,NaN
...,...,...,...,...,...,...
36078,whoknows,Musiq,2004,65,13,NaN
36079,wwwmemory,Alan Jackson,2000,45,9,NaN
36080,wwwmemory,Alan Jackson,2001,45,15,NaN
36081,¿Dònde Està Santa Claus? (Where Is Santa Claus?),Augie Rios,1958,47,3,NaN


In [45]:
unique_billboard_tracks_df[unique_billboard_tracks_df['song'].str.contains("-")]

,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1
10,(1-2-3-4-5-6-7) Count The Days,Inez & Charlie Foxx,1968,76,5,NaN
63,(How I Spent My Summer Vacation) Or A Day At T...,Cheech & Chong,1975,54,5,NaN
157,(Not Just) Knee Deep - Part 1,Funkadelic,1979,77,4,NaN
174,(See You At The) Go-Go,Dobie Gray,1965,69,7,NaN
184,(Shu-Doo-Pa-Poo-Poop) Love Being Your Fool,Travis Wammack,1975,38,10,NaN
...,...,...,...,...,...,...
35944,Youre The One-Part II,Little Sister,1970,22,11,NaN
36026,Zazueira (Za-zoo-wher-a),Herb Alpert & The Tijuana Brass,1969,78,5,NaN
36028,Zero-Zero,Lawrence Welk,1962,98,1,NaN
36032,Zip-A-Dee Doo-Dah,Bob B. Soxx And The Blue Jeans,1962,9,7,NaN


In [54]:
# SPOTIFY API
# Import dependencies for Spotipy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

# Store client id and secret key strings
cid = '3865da35fc744998a8d5a1c988cf3247'
secret = 'e50f85a85aaf4a798bbd7e386b1ee106'

# Create objects for accessing Spotify API
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [52]:
# Create function to retrieve track ID from Spotify given the artist and song title
def get_track_id(song_title, artist):
    # Select first artist if multiple listed with "Featuring" keyword
    if 'Featuring' in artist:
        artist = artist.split(' Featuring ')[0]
    # Select first artist if multiple listed with "with" keyword
    elif ' with ' in artist:
        artist = artist.split(' with ')[0]
    elif ' With ' in artist:
        artist = artist.split(' With ')[0]
    elif "," in artist:
        artist = artist.split(',')[0]
    # Make API call
    try:
        # Store search results for track with artist and song title as query parameters
        searchResults = sp.search(q=f"artist:{artist} track:{song_title}", type="track", limit=1)
        # Store track ID
        track_id = searchResults['tracks']['items'][0]['id']
    
        return track_id
    
    # Executes if no match found yet
    except:
        # Check for '&' character
        if ' & ' in artist:
            try:
                # Slice artist name before '&' symbol to get first artist name
                artist = artist.split(' & ')[0]
                # Store search results for track with artist and song title as query parameters
                searchResults = sp.search(q=f"artist:{artist} track:{song_title}", type="track")
                # Store track ID
                track_id = searchResults['tracks']['items'][0]['id']

                return track_id
            
            # Executes if still no match found
            except:
                # Print song title and artist for non-match
                print(f'No ID found for {song_title} by {artist}')

In [48]:
def get_audio_features(uri):
    searchResults = sp.audio_features(uri)
    return searchResults

In [55]:
song = "Aquemini"
artist = "Outkast"
# sp.search(q=f"track:{song} artist:{artist}", type='track', limit=1)
get_track_id(song, artist)

In [23]:
unique_billboard_tracks_df[unique_billboard_tracks_df['song']==song]

,song,artist,year,peak-rank,weeks-on-board,weeks-at-no1
11164,Harper Valley PTA,Jeannie C. Riley,1968,1,13,NaN
11165,Harper Valley PTA,King Curtis & The Kingpins,1968,93,2,NaN


In [71]:
# Import dependency for swifter module
import swifter


billboard_1960s = billboard_1960s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1960s.to_csv(f'./Resources/billboard_1960s.csv')

In [ ]:
billboard_1970s = billboard_1970s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1970s.to_csv(f'./Resources/billboard_1970s.csv')


billboard_1980s = billboard_1980s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1980s.to_csv(f'./Resources/billboard_1980s.csv')


billboard_1990s = billboard_1990s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_1990s.to_csv(f'./Resources/billboard_1990s.csv')


billboard_2000s = billboard_2000s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2000s.to_csv(f'./Resources/billboard_2000s.csv')


billboard_2010s = billboard_2010s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2010s.to_csv(f'./Resources/billboard_2010s.csv')


billboard_2020s = billboard_2020s[['song', 'artist']].swifter.apply(lambda row:get_track_id(row.song,row.artist),axis=1)
billboard_2020s.to_csv(f'./Resources/billboard_2020s.csv')